<font size=5 color="blue">Library and Data Import</font>

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import words as w
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
import json
from sklearn.model_selection import train_test_split
from Data_Cleaning import datacleaning, datamapping, datapreprocessing
from nltk.stem import LancasterStemmer, PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy
from collections import Counter
# from NLP import NLP as nlp

ModuleNotFoundError: No module named 'Data_Cleaning'

In [2]:
initialize = True
# lemmatization
if initialize:
    nltk.download('words')
tickets = pd.read_excel('Ticket Details.xlsx', header=1 , sheet_name = "Sheet 1", names=['Ticketid', 'Calendar Week', 'Ticket Created Time', 'Priority', 'Site', 'Problem Description', 'General Category', 'Sub Category', 'Resolution', 'Customer Department', 'Predicted General Category', 'Predicted Sub Category'])
#tickets.columns = tickets.columns.str.strip()
tickets = pd.DataFrame([tickets['Ticketid'], tickets['Problem Description']]).transpose()
tickets = tickets.fillna('')

[nltk_data] Downloading package words to /ifxhome/manna/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
##extraction of problem description
# regex_remove_punctuations = r'[~`!@#\$%\^&\*\(\)_\+\-\=\{\}\|\[\]\\:;"\'<>\?,\.\/]'
# for i in range(tickets.shape[0]):
#     a=re.sub(r'(\([2-9]\))[\s?[a-zA-Z]\W*\D*\S*]*', '',tickets.loc[i]['Problem Description'])
#     b = re.sub(r'(Lot Number:)(\s*[a-zA-Z]*\W*\D*\S*)*','', a)
#     c = re.sub(r'(\(1\))(\s*[a-zA-Z]*\W*\D*\S*)(error\))','', b)
#     text = c.replace('\n', ' ')
#     text = text.replace('\t', ' ')
#     text = text.replace(r'(\'*\')', '')
#     text = text.strip()
# #     text = re.sub(regex_remove_punctuations,' ', text)
#     text = text.lower()
#     tickets.loc[i]['Problem Description'] = text

<font size=5> Initial data cleaning</font>

In [4]:
fileref = 'lookup.xlsx'
cleaned = datacleaning(tickets)
processed = datamapping(cleaned,fileref)
# processed = datapreprocessing(processed)

In [5]:
## focusing on the problem description
processed= pd.DataFrame([processed['Ticketid'], processed['Desc']]).transpose()

In [6]:
processed.dropna(subset=['Desc'], inplace=True)
# processed

In [7]:
from collections import Counter

# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [8]:
text = processed.Desc

counter = counter_word(text)

In [9]:
num_words = len(counter)

# Max number of words in a sequence
max_length = 20

In [11]:
processed.reset_index(inplace=True)
processed.drop("level_0", inplace=True, axis=1)
processed.drop("index", inplace=True, axis=1)
processed

,Ticketid,Desc
0,INC000004851418,the fajob does not belong to this equipment
1,INC000004851509,error during trackout camstar total ndpw item ...
2,INC000004852100,cannot move out lot strip map total quantity
3,INC000004852981,missing next step
4,INC000004853010,the target step could not be determined for th...
...,...,...
8699,INC000006004547,user request to raise camstar it ticket user i...
8700,INC000006004548,buttons are grey out
8701,INC000006004924,original quantityity of the lot to be tracked ...
8702,INC000006005058,lot is not allowed for


In [12]:
## need to do it at the beginning
# import nltk
# nltk.download('averaged_perceptron_tagger')
def remove_uninfo_word(sen):
    tokens = nltk.word_tokenize(sen)
    tagged = nltk.pos_tag(tokens)
    res = ["".join(tups[0]) for tups in tagged if not re.match(r"^VBP$", tups[1]) and not re.match(r"^WDT$", tups[1]) and not re.match(r"^VBD$", tups[1]) \
                                                         and not re.match(r"^CC$", tups[1]) and not re.match(r"^DT$", tups[1]) \
                                                        and not re.match(r"^VBZ$", tups[1]) and not re.match(r"^com$", tups[0]) and\
                                                        not re.match(r"^MD$", tups[1]) and not re.match(r"^IN$", tups[1]) \
                                                           and not re.match(r"^TO$", tups[1]) and not re.match(r"^PRP$", tups[1]) \
                                                           and not re.match(r"^user$", tups[0]) and not re.match(r"^infineon$", tups[0])
                                                        and not re.match(r"^lot$", tups[0]) and not re.match(r"^does$", tups[0])\
                          and not re.match(r"^did$", tups[0]) and not re.match(r"^didn't$", tups[0]) and not re.match(r"^camstar$", tups[0])]
#     print(tagged)
    return " ".join(res)

In [13]:
lemmatized=processed.copy()
lemmatized["desc"]=""
lemmatized["desc"] = lemmatized.Desc.map(lambda x: remove_uninfo_word(x))
lancaster = PorterStemmer()
lem = WordNetLemmatizer()
for i in range(lemmatized.shape[0]):
    text_list = lemmatized["desc"].loc[i].split(" ")
#     text_list = [lancaster.stem(word) for word in text_list]
    text_list = [lem.lemmatize(word) for word in text_list]
    text_list = [lem.lemmatize(word, 'v') for word in text_list]
    text_list = [lem.lemmatize(word, 'a') for word in text_list]
    text_list = [lem.lemmatize(word, 'r') for word in text_list]
    text_list = [lem.lemmatize(word, 's') for word in text_list]
    lemmatized["desc"].loc[i]= " ".join(text_list)
# lemmatized["Desc"] = lemmatized.Desc.map(lambda x: lancaster.stem(x))
# lemmatized["Desc"] = lemmatized.Desc.map(lambda x: lemmatizer.lemmatize(x))
# lemmatized = processed.copy()
# processed[:100].to_excel("new_ticket.xlsx")

<font size=3 color="grey">Just for trial</font>

In [ ]:
tokens = nltk.word_tokenize(processed["Desc"].loc[26])
tagged = nltk.pos_tag(tokens)
res1=""
res = ["".join(tups[0]) for tups in tagged if not re.match(r"^VBP$", tups[1]) and not re.match(r"^WDT$", tups[1]) and not re.match(r"^VBD$", tups[1]) \
                                                         and not re.match(r"^CC$", tups[1]) and not re.match(r"^DT$", tups[1]) \
                                                        and not re.match(r"^VBZ$", tups[1]) and not re.match(r"^com$", tups[0]) and\
                                                        not re.match(r"^MD$", tups[1]) and not re.match(r"^IN$", tups[1]) \
                                                           and not re.match(r"^TO$", tups[1]) and not re.match(r"^PRP$", tups[1]) \
                                                           and not re.match(r"^user$", tups[0]) and not re.match(r"^infineon$", tups[0])
                                                        and not re.match(r"^lot$", tups[0]) and not re.match(r"^does$", tups[0])\
                          and not re.match(r"^did$", tups[0]) and not re.match(r"^didn't$", tups[0]) and not re.match(r"^camstar$", tups[0])]
print(tagged)
" ".join(res)

In [ ]:
processed[processed["Ticketid"]=="INC000004855678"]["Desc"]

In [ ]:
# print(processed.Desc[0])
# print(processed_seq[0])
# reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# def decode(text):
#     return " ".join([reverse_word_index.get(i, "?") for i in text])
# decode(processed_seq[0])

In [ ]:
# tickets_new = pd.read_excel('new_ticket_me.xlsx')
# tickets_new = tickets_new.dropna()
# tickets_new['label']=tickets_new['Label'].astype('int')
# tickets_new=pd.DataFrame([tickets_new['Desc'], tickets_new['label']]).transpose()
# (xtrain,xtest,ytrain,ytest)=train_test_split(tickets_new["Desc"], tickets_new["label"], test_size=0.2)


In [ ]:
# from keras.preprocessing.text import Tokenizer

# tokenizer = Tokenizer(num_words=num_words)
# tokenizer.fit_on_texts(xtrain)
# word_index = tokenizer.word_index
# processed_seq = tokenizer.texts_to_sequences(xtrain)
# from keras.preprocessing.sequence import pad_sequences

# processed_pad = pad_sequences(
#     processed_seq, maxlen=max_length, padding="post", truncating="post"
# )
# test_sequences = tokenizer.texts_to_sequences(xtest)
# test_padded = pad_sequences(
#     test_sequences, maxlen=max_length, padding="post", truncating="post"
# )

In [ ]:
# from keras.models import Sequential
# from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, BatchNormalization
# from keras.initializers import Constant
# from tensorflow.keras.optimizers import Adam

# model = Sequential()

# model.add(Embedding(num_words, 32, input_length=max_length))
# model.add(LSTM(64, dropout=0.1))
# model.add(BatchNormalization())
# model.add(Flatten())
# model.add(Dense(18, activation="sigmoid"))


# optimizer = Adam(learning_rate=4e-4)

# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
# max_length

In [ ]:
# model.summary()

In [ ]:
# y_train = np.asarray(ytrain).astype('int64')
# y_test = np.asarray(ytest).astype('int64')

##not important one
# from sklearn import preprocessing
# le= preprocessing.LabelEncoder()
# le.fit(y_train)
# y_train =  le.fit_transform(y_train)
# y_test = le.fit_transform(y_test)

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# history = model.fit(
#     processed_pad, tf.convert_to_tensor(y_train), batch_size=32, epochs=10, validation_data=(test_padded, tf.convert_to_tensor(y_test)),
# )

In [ ]:
# model.evaluate(test_padded[:3], y_test[:3])
# test_padded.shape

In [ ]:
# y_test

<font size="5" color="red">Spell Correction</font>

<font color="blue">Created a manual dictionary for the wrong words which i have explored so far. New words can be added later on.</font>

In [ ]:
# ##spell correction step
# from nltk.stem import LancasterStemmer
# m_dict={"abrcode":"barcode","msg":"message","mesg":"message", "pwd":"password","num":"number", "dn":"delivery note", "promopt":"prompt",\
#        "sucess":"success","wefer": "wafer","aldy":"already","quatit":"quantity",\
#        "qty":"quantity","qtys":"quantity","quanity":"quantity","buttos":"buttons","buttin":"buttons",\
#        "pls":"please","pelase":"please","txn":"transaction","tryin":"trying","tring":"trying",\
#        "remot":"remote","exitsts":"exists","previos":"previous","mvin":"movein","alowe":"allowed","cariier":"carrier",\
#        "coloumn":"column","produst":"product","nto":"not","complate":"complete","complet":"complete","selclected":"selected","beccause":"because","continu":"continue",\
#        "atributes":"attributes","unabe":"unable","unble":"unable","erorr":"error","movi":"move",\
#        "gray":"grey","greyed":"grey"}

In [ ]:
# from nltk.stem import LancasterStemmer
# from nltk.stem import WordNetLemmatizer 

In [ ]:
# lancaster = LancasterStemmer()
# lemmatizer = WordNetLemmatizer()
# for i in range(tickets.shape[0]):
#     new_l = tickets["Problem Description"].loc[i].split()
#     for j, val in enumerate(new_l):
#         if val in m_dict:
# #             print(val)
#             new_l[j]=m_dict[val]
#         new_l[j]=lancaster.stem(new_l[j])
#         new_l[j]= lemmatizer.lemmatize(new_l[j])
#     tickets["Problem Description"].loc[i]=" ".join(new_l)

<font color="blue"> A example of the correction is that the below sentence had mesg in the original problem description and now the 
message is replaced with the wrong one </font>

In [ ]:
# tickets["Problem Description"].loc[99]

In [ ]:
# tickets[tickets["Ticketid"]=="INC000004861950"]

In [ ]:
# tickets[:101].to_excel("tickets.xlsx")

In [ ]:
# tickets_new = pd.read_excel('tickets.xlsx', names=["Ticketid","Desc", "label"])
# tickets_new = tickets_new.dropna()
# tickets_new['label']=tickets_new['label'].astype('int')
# tickets_new=pd.DataFrame([tickets_new['Desc'], tickets_new['label']]).transpose()

<font size=5 color="green">Finding important words</font>

<font color="red"> I am trying to find some potential problem words for word embedding</font>

In [ ]:
# tickets['json']=""
# tickets.dropna(subset=['Desc'], inplace=True)

In [ ]:
# from spacy.tokens import Doc
# from spacy.training import Example
# import spacy
# from spacy.matcher import PhraseMatcher
# # import nlp

In [ ]:
# # tickets.loc[22]['json']
# l = ["grey","out", "button","gray", "does","not","belong", "mismatch","does", "its", "lot","qty",
#     "match","qty","quantity", "sequencing","fail","missing","next","step", "could","not", "be","determined"]
# tickets1=pd.DataFrame(columns=['Ticketid', 'Problem Description', 'json'])
# for i in range(tickets.shape[0]):
#     tickets['json'].loc[i]=[]
#     for k in l:
#         print(k)
#         if k in tickets['Desc'].loc[i]:
#             if len(tickets['json'].loc[i])==0 and len(tickets['Desc'].loc[i])!=0:
# #                 tickets['json'].loc[i]['text']=tickets['Problem Description'].loc[i]
# #                 start = tickets['Problem Description'].loc[i].find(k)
# #                 end = start + len(k)
# #                 l1=(start, end, "PROBLEM")
#                 tickets['json'].loc[i]=[k]
#             elif len(tickets['json'].loc[i])!=0 and len(tickets['Desc'].loc[i])!=0:
# #                 start = tickets['Problem Description'].loc[i].find(k)
# #                 end = start + len(k)
# #                 l1=(start, end, "PROBLEM")
#                 tickets['json'].loc[i].append(k)
#     if len(tickets['json'].loc[i])==0:
#         tickets['json'].loc[i]=""
# #     if len(tickets['json'].loc[i])!=0:
# #         tickets['json'].loc[i]=json.dumps(list(tickets['json'].loc[i]))

In [ ]:
# tickets.reset_index(inplace=True)
# tickets

In [ ]:
# tickets1=tickets.copy()
# tickets1.dropna(subset=['json'], inplace=True)
# tickets.to_excel('tickets.xlsx')

In [15]:
lemmatized.reset_index(inplace=True)
nlp = spacy.load("en_core_web_sm")
lemmatized["common_words"]=""
for i in range(lemmatized.shape[0]):
    doc = nlp(lemmatized['desc'].loc[i])
    # all tokens that arent stop words or punctuations
    words = [token.text
             for token in doc
             if not token.is_stop and not token.is_punct]
    # five most common tokens
    word_freq = Counter(words)
    common_words = word_freq.most_common()

    lemmatized["common_words"].loc[i]=[i[0] for i in common_words]
    lemmatized["common_words"].loc[i]=" ".join(lemmatized["common_words"].loc[i])
lemmatized.to_excel("tickets.xlsx")

/home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# lNonDicW=[]
# # english_words = set(nltk.corpus.words.words())
# from nltk.corpus import wordnet
# for i in range(tickets.shape[0]):
#     doc = nlp(tickets['Problem Description'].loc[i])
#     for token in doc:
#         if not wordnet.synsets(token.text) and not token.is_stop and not token.is_punct:
#             lNonDicW.append((token.text,tickets['Problem Description'].loc[i]))

In [ ]:
# l2=[]
# l_badwords=['qty', 'camstar', 'infineon', 'com','boc','admin']
# for i in lNonDicW:
#     if re.match(r'\s+', i[0]) or re.match(r'\w*\d+',i[0]) or i[0] in l_badwords:
#         pass
#     elif re.match(r'[a-z]+', i[0]):
#         l2.append(i)

In [ ]:
# pd.DataFrame(l2).to_excel("nondictwords.xlsx")

<font size=5 color="red">WordNet part</font>

In [1]:
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn

In [20]:
# !pip install PyStanfordDependencies

In [56]:
from bs4 import BeautifulSoup as bs
import requests
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from collections import Counter

import matplotlib.pylab as plt
plt.rcParams['figure.dpi'] = 150


In [76]:
lemmatized.isna().sum()

index           0
Ticketid        0
Desc            0
desc            0
common_words    0
dtype: int64

In [81]:
processed['Desc'].describe()

count                                            8704
unique                                           5995
top       the fajob does not belong to this equipment
freq                                              401
Name: Desc, dtype: object

In [77]:
def generate_N_grams(text,ngram=1):
    words=[word for word in text.split(" ") if word not in set(stopwords.words('english'))]  
    print("Sentence after removing stopwords:",words)
    temp=zip(*[words[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    return ans

In [80]:
generate_N_grams(processed.loc[126]['Desc'],3)

Sentence after removing stopwords: ['trackin', 'trackout', 'move', 'button', 'grey', 'fajob', 'transaction']


['trackin trackout move',
 'trackout move button',
 'move button grey',
 'button grey fajob',
 'grey fajob transaction']